In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import RMSprop
from matplotlib import pyplot as plt
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing import image_dataset_from_directory
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        os.path.join(dirname, filename)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from sklearn.model_selection import train_test_split
df = pd.read_csv('/kaggle/input/plant-pathology-2021-fgvc8/train.csv')
submission = pd.read_csv('../input/plant-pathology-2021-fgvc8/sample_submission.csv')
train, val = train_test_split(df, test_size=0.2)

Reads in the the csv to pandas dataframe. 80% of the data is used for training and 20% used for validation.

In [ ]:
plt.imshow(np.array(img_to_array(load_img('../input/plant-pathology-2021-fgvc8/train_images/800113bb65efe69e.jpg')).astype("uint8")))

The first image from the training set

In [ ]:
plt.imshow(np.array(img_to_array(load_img('../input/plant-pathology-2021-fgvc8/train_images/800113bb65efe69e.jpg',target_size=(128,128))).astype("uint8")))

The first image of the training set resiszed to 128x128, because the raw images are very large and take a long time to load and process.

The image data generator will resize the images and load them as needed for the model to train.

In [ ]:
image_datagen = ImageDataGenerator()

train_ds = image_datagen.flow_from_dataframe(
    dataframe = train,
    directory = '../input/plant-pathology-2021-fgvc8/train_images',
    x_col = "image",
    y_col = "labels",
    target_size = (128,128),
    class_mode='categorical',
    batch_size = 64,
    shuffle = True,
    seed = 9,
    validate_filenames = False,
)

val_ds = image_datagen.flow_from_dataframe(
    dataframe = val,
    directory = '../input/plant-pathology-2021-fgvc8/train_images',
    x_col = "image",
    y_col = "labels",
    target_size = (128,128),
    class_mode='categorical',
    batch_size = 64,
    shuffle = True,
    seed = 9,
    validate_filenames = False
)
test_ds = image_datagen.flow_from_dataframe(
    dataframe=submission,
    directory='../input/plant-pathology-2021-fgvc8/test_images',
    x_col = "image",
    y_col = "labels",
    target_size = (128,128),
    class_mode='categorical',
    validate_filenames=False)

Convolutional networks are the most used machine leanring type for image classification

In [ ]:
model = Sequential([
  layers.experimental.preprocessing.Rescaling(1./255, input_shape=(128, 128, 3)),
  layers.Conv2D(64, (3,3), padding='same', activation='relu'),
  layers.MaxPooling2D((2,2)),
  layers.Conv2D(64, (3,3), padding='same', activation='relu'),
  layers.MaxPooling2D((2,2)),
  layers.Conv2D(64, (3,3), padding='same', activation='relu'),
  layers.MaxPooling2D((2,2)),
  layers.Flatten(),
  layers.Dense(12, activation='softmax')
])

The optimizer adam is also the most used type of optimizer for image classifcation. The loss function categorical crossentropy is a used for multi-class classification problems.

In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

Early stopping stops the model from training if the loss function on the validation data is not improving. Early stopping can help prevent over fitting and waisted time training.

In [ ]:
early_stopping = EarlyStopping(monitor = 'val_loss',min_delta=.01,patience=3,restore_best_weights=True)

In [ ]:
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=10,
  callbacks=early_stopping,
  steps_per_epoch=100,
  validation_steps=20,
  max_queue_size=1000,
  use_multiprocessing=True,
  workers=4
)

Shows the accuracy on the training data and validation data over each epoch

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.show()

In [ ]:
def get_key(val_array, my_dict):
    keys = []
    for val in val_array:
        for key, value in my_dict.items():
             if val == value:
                keys.append(key)
    return keys

In [ ]:
submission['labels'] = get_key(np.argmax(model.predict(test_ds), axis=1), train_ds.class_indices)

In [ ]:
model.save('./plant_pathology_cnn')

In [ ]:
%cd /kaggle/working
from IPython.display import FileLink as FileLink(r'*plant_pathology_cnn*')